In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import sys
import os
import psutil
import gc

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir + '/valid', transform=valid_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
num_classes = len(cat_to_name.keys())
print('number of categories: {}'.format(num_classes))

number of categories: 102


In [5]:
device = torch.device('cpu')
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = models.vgg16(pretrained=True)
    
    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False

    classifier_dict = checkpoint['classifier']
    model.classifier = nn.Sequential(classifier_dict)
    model.load_state_dict(checkpoint['state_dict'])    
    del(checkpoint)
    
    return model

model = load_checkpoint('checkpoint.pth')
print(model)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

#model.classifier = nn.Sequential(classifier_dict)
#criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load model into gpu
model.to(device);

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [6]:
def memReport():
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            print(type(obj), obj.size())
    
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)


In [7]:
def validate_nn(loader, model):
    test_loss = 0
    accuracy = 0
    model.eval()
    steps = 0
    with torch.no_grad():
        for inputs, labels in loader:
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        model.train()
        return test_loss, accuracy# Do validation on the test set

In [8]:
#loss, accuracy = validate_nn(testloader, model)
#print(f"Test loss: {loss/len(testloader):.3f}, "
#      f"Test accuracy: {accuracy/len(testloader):.3f}")

In [9]:
def train_nn(model, epochs, criterion, optimizer, trainloader, validloader):
    running_loss = 0
    print_every = 5
    model.train()
    for epoch in range(epochs):
        #cpuStats()
        #memReport()
        steps = 0
        for inputs, labels in trainloader:
            steps += 1
            
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:

                valid_loss, accuracy = validate_nn(validloader, model)                    
                print(f"Step {steps}/{len(trainloader)}.. ",
                      f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Valid. loss: {valid_loss/len(validloader):.3f}.. "
                      f"Valid. accuracy: {accuracy/len(validloader):.3f}")

                running_loss = 0
                model.train()            

In [10]:
epochs = 4
torch.cuda.empty_cache()
train_nn(model, epochs, criterion, optimizer, trainloader, validloader)                

3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
18.7
svmem(total=16978071552, available=9275428864, percent=45.4, used=7702642688, free=9275428864)
memory GB: 1.6404304504394531
<class 'torch.nn.parameter.Parameter'> torch.Size([256, 256, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([256, 256, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 256, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([512])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 512, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([512])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 512, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([512])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 512, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([512])
<class 'torch.nn.parameter.Parameter

Step 185/205..  Epoch 1/4.. Train loss: 1.655.. Valid. loss: 1.513.. Valid. accuracy: 0.663
Step 190/205..  Epoch 1/4.. Train loss: 2.225.. Valid. loss: 1.695.. Valid. accuracy: 0.627
Step 195/205..  Epoch 1/4.. Train loss: 2.479.. Valid. loss: 1.532.. Valid. accuracy: 0.635
Step 200/205..  Epoch 1/4.. Train loss: 1.914.. Valid. loss: 1.492.. Valid. accuracy: 0.643
Step 205/205..  Epoch 1/4.. Train loss: 2.258.. Valid. loss: 1.530.. Valid. accuracy: 0.633
3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
19.2
svmem(total=16978071552, available=8493535232, percent=50.0, used=8484536320, free=8493535232)
memory GB: 2.366363525390625
<class 'torch.Tensor'> torch.Size([4096, 25088])
<class 'torch.Tensor'> torch.Size([4096, 25088])
<class 'torch.Tensor'> torch.Size([4096])
<class 'torch.Tensor'> torch.Size([4096])
<class 'torch.Tensor'> torch.Size([102, 4096])
<class 'torch.Tensor'> torch.Size([102, 4096])
<class 'torch.Tensor'> torch.Size([102])
<class 'to

Step 130/205..  Epoch 2/4.. Train loss: 2.290.. Valid. loss: 1.391.. Valid. accuracy: 0.636
Step 135/205..  Epoch 2/4.. Train loss: 2.183.. Valid. loss: 1.426.. Valid. accuracy: 0.638
Step 140/205..  Epoch 2/4.. Train loss: 1.943.. Valid. loss: 1.589.. Valid. accuracy: 0.620
Step 145/205..  Epoch 2/4.. Train loss: 2.285.. Valid. loss: 1.546.. Valid. accuracy: 0.629
Step 150/205..  Epoch 2/4.. Train loss: 2.447.. Valid. loss: 1.592.. Valid. accuracy: 0.604
Step 155/205..  Epoch 2/4.. Train loss: 2.269.. Valid. loss: 1.540.. Valid. accuracy: 0.635
Step 160/205..  Epoch 2/4.. Train loss: 1.978.. Valid. loss: 1.503.. Valid. accuracy: 0.628
Step 165/205..  Epoch 2/4.. Train loss: 1.894.. Valid. loss: 1.437.. Valid. accuracy: 0.645
Step 170/205..  Epoch 2/4.. Train loss: 1.972.. Valid. loss: 1.488.. Valid. accuracy: 0.645
Step 175/205..  Epoch 2/4.. Train loss: 1.957.. Valid. loss: 1.495.. Valid. accuracy: 0.637
Step 180/205..  Epoch 2/4.. Train loss: 1.704.. Valid. loss: 1.523.. Valid. accu

Step 75/205..  Epoch 3/4.. Train loss: 1.964.. Valid. loss: 1.304.. Valid. accuracy: 0.673
Step 80/205..  Epoch 3/4.. Train loss: 2.075.. Valid. loss: 1.286.. Valid. accuracy: 0.671
Step 85/205..  Epoch 3/4.. Train loss: 2.563.. Valid. loss: 1.400.. Valid. accuracy: 0.652
Step 90/205..  Epoch 3/4.. Train loss: 2.262.. Valid. loss: 1.399.. Valid. accuracy: 0.641
Step 95/205..  Epoch 3/4.. Train loss: 1.930.. Valid. loss: 1.518.. Valid. accuracy: 0.632
Step 100/205..  Epoch 3/4.. Train loss: 2.281.. Valid. loss: 1.363.. Valid. accuracy: 0.673
Step 105/205..  Epoch 3/4.. Train loss: 1.896.. Valid. loss: 1.300.. Valid. accuracy: 0.686
Step 110/205..  Epoch 3/4.. Train loss: 1.938.. Valid. loss: 1.265.. Valid. accuracy: 0.676
Step 115/205..  Epoch 3/4.. Train loss: 1.978.. Valid. loss: 1.287.. Valid. accuracy: 0.679
Step 120/205..  Epoch 3/4.. Train loss: 2.287.. Valid. loss: 1.355.. Valid. accuracy: 0.692
Step 125/205..  Epoch 3/4.. Train loss: 2.169.. Valid. loss: 1.451.. Valid. accuracy:

Step 20/205..  Epoch 4/4.. Train loss: 1.611.. Valid. loss: 1.298.. Valid. accuracy: 0.687
Step 25/205..  Epoch 4/4.. Train loss: 1.879.. Valid. loss: 1.333.. Valid. accuracy: 0.675
Step 30/205..  Epoch 4/4.. Train loss: 2.067.. Valid. loss: 1.244.. Valid. accuracy: 0.669
Step 35/205..  Epoch 4/4.. Train loss: 2.159.. Valid. loss: 1.253.. Valid. accuracy: 0.672
Step 40/205..  Epoch 4/4.. Train loss: 1.883.. Valid. loss: 1.255.. Valid. accuracy: 0.668
Step 45/205..  Epoch 4/4.. Train loss: 1.872.. Valid. loss: 1.221.. Valid. accuracy: 0.688
Step 50/205..  Epoch 4/4.. Train loss: 1.742.. Valid. loss: 1.199.. Valid. accuracy: 0.698
Step 55/205..  Epoch 4/4.. Train loss: 1.908.. Valid. loss: 1.225.. Valid. accuracy: 0.700
Step 60/205..  Epoch 4/4.. Train loss: 1.925.. Valid. loss: 1.230.. Valid. accuracy: 0.707
Step 65/205..  Epoch 4/4.. Train loss: 1.518.. Valid. loss: 1.251.. Valid. accuracy: 0.714
Step 70/205..  Epoch 4/4.. Train loss: 2.031.. Valid. loss: 1.314.. Valid. accuracy: 0.697

In [11]:
loss, accuracy = validate_nn(testloader, model)
print(f"Test loss: {loss/len(testloader):.3f}, "
      f"Test accuracy: {accuracy/len(testloader):.3f}")

Test loss: 1.264, Test accuracy: 0.691
